# Intersubject Correlation: Synchronizing our Social Cognition
**Summary**: In this project, [we](http://scraplab.org/) are interested in examining how neural synchrony among participants may be elicited for different categories of social stimuli. For instance, will we see an increase in neural synchrony when there the number of characters present on-screen increases? In addition to the visual aspect--participants, as a part of the [Naturalistic Neuroimaging Database](https://www.naturalistic-neuroimaging-database.org/) (NNDb) paradigm are watching one of ten movies--we also consider some elements of speech, such as when overlap occurs, or how emotional the speech is. 

First, we install the packages used in this stage of the analysis. We also use several deep learning packages to annotate various features of the NNDb movies, which we cite as the data is incorporated into the pipeline. 

In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
datapath='/Volumes/Scraplab/data/ds002837/derivatives/'
isc_outs = '/Volumes/Scraplab/psypose_fmri/isc_analysis/'

#Generate the subject list and filenames
func_data = os.listdir(datapath)
sub_ids = [x for x in func_data if ('sub-') in x] #grab all the subject IDs for easy filtering

#net together all the datafiles independent of which task they are from
all_task_subs = [] 
for id in sub_ids:
    all_task_subs.append(glob.glob(os.path.join(datapath+id+'/func/*blur_censor_ica.nii.gz'))[0])

#Listing the 10 different movie stimuli 
tasknames = ['12yearsaslave','500daysofsummer','backtothefuture','citizenfour',
           'littlemisssunshine', 'pulpfiction','split','theprestige',
           'theshawshankredemption','theusualsuspects']

## Summing the functional timecourses

First, for each movie, we sum up all the participants' fMRI timecourses, creating one giant matrix. This is so that later on in the pipeline we can subtract out a single subject of the group, and then correlate that single subject's timecourse with the summed timecourse of the rest of the participants.

In [ ]:
for task in tasknames:
    #retrieve the task's nii files
    task_files = [x for x in all_task_subs if task in x]
    one_subj = nb.load(task_files[0]) #pull a single subject to grab the film length (the # of TRs)
    film_length = one_subj.shape[3] #grab that 3rd dimension, the TR
    
    #create an empty matrix the size of all the subject timecourses 
    group_brain = np.empty([len(task_files),64,76,64,film_length])
    index=0
    
    #now we go into the list of the current movie's participants 
    for file in task_files:  
        one_nii = nb.load(file) #load each file
        group_brain[index,:,:,:,:] = np.array(one_nii.get_fdata(),dtype=np.float64) # retrieve the functional data and 'index it' into the location that corresponds to the subject
        index += 1 #update the index 
    
    #this loop takes about 50 minutes per movie. 
    group_brain_sum = group_brain.sum(axis=0)
    #affix header and affine data to the averaged nifti file
    array_img = nb.Nifti1Image(group_brain_sum,one_subj.affine,one_subj.header)
    nb.save(array_img,"isc_analysis/"+task+"/"+task+"_summed_timecourse.nii.gz")

## Creating a Mask for our Data
We want to mask our data so that we don't inadvertently correlate data outside of the boundaries of the brain - not only is this unecessary, it can also cause a number of issues further on in our analysis.

In [ ]:
global_mask_list = []

for subject in all_task_subs: 
    subject_nb = nb.load(subject)
    subject_arr = subject_nb.get_fdata()
    subject_mask = np.mean(subject_arr != 0, axis=3) 
    global_mask_list.append(subject_mask) #subject_mask shape: (64,76,64)

np.save(isc_outs+'global_mask_list', global_mask_list)

In [ ]:
mask_np = np.load(isc_outs+'global_mask_list.npy')
global_mask = mask_np != 0
global_mask_med = np.median(global_mask,axis=0)
np.save(isc_outs+'global_mask', global_mask_med)

## Correlating Subject Data
Now we are onto the correlation itself. For each movie, we load the summed data that we created earlier, and subtract out one subject. We take that subject's data and correlate it with the remaining participants' data. We perform this correlation as a sliding window-every 20 seconds we take a new correlation, and we do this throughout the entire length of the movie. This is a very computationally intensive process, so we advise you to parallelize this stage of the pipeline. 

In [ ]:
mask = np.load(isc_outs+'global_mask.npy')

for task in tasknames: 
    subject_list = [x for x in func_data if task in x]
    summed_timecourse_nb = nb.load(isc_outs+task+'/'+task+'_summed_timecourse.nii.gz')
    summed_timecourse_arr = summed_timecourse_nb.get_fdata()
    movie_tr_length = summed_timecourse_nb.shape[3]
    inter_subject_correlation = np.empty(summed_timecourse_nb.shape)

    for i in subject_list:
        sub_id = i.split("/")[6]
        sub_nb = nb.load(i)
        sub_arr = sub_nb.get_fdata()
        xn, yn, zn = sub_arr.shape[0], sub_arr.shape[1], sub_arr.shape[2]
        header, affine = sub_nb.header, sub_nb.affine
        group_data = summed_timecourse_arr - sub_arr

        for tr in range(movie_tr_length):
            single_subject_window = sub_arr[:,:,:,tr:tr+20]
            group_data_window = group_data[:,:,:,tr:tr+20]

            for x in range(xn):
                for y in range(yn):
                    for z in range(zn):
                        if mask[x,y,z]:
                            inter_subject_correlation[x,y,z,tr] = np.corrcoef(single_subject_window[x,y,z].flatten(),group_data_window[x,y,z].flatten(), rowvar=False)[0,1]
                        else:
                            continue

        array_img = nb.Nifti1Image(inter_subject_correlation,affine,header)
        nb.save(array_img,isc_outs+task+"/"+task+"_"+sub_id+"_correlated_timeseries.nii.gz")

## Averaging ISC Data

Next, we take the average of each movie's ISC data. We will model this data with a general linear model, as shown in the notebook isc-modeling. 

In [ ]:
for task in tasknames: 
    isc_dir = os.listdir(isc_outs+task) 
    all_isc_files = [x for x in isc_dir if ('correlated_timeseries.nii.gz') in x]
    subject_iscs = [x for x in all_isc_files if task in x]
    
    #create an empty matrix the size of all the subject timecoursess
    dim_sub = nb.load('/Volumes/Scraplab/psypose_fmri/isc_analysis/'+task+os.sep+"".join(all_subject_iscs[0:1]))
    isc_data_arr = np.empty([len(all_subject_iscs),dim_sub.shape])
    index = 0
    
    for subject in subject_iscs:
        subject_nb = nb.load('/Volumes/Scraplab/psypose_fmri/single_subject_correlations/'+subject)
        isc_data_arr[index,:,:,:,:] = np.array(subject_nb.get_fdata(),dtype=np.float64)
        index += 1
    
    avg_isc = isc_data_arr.mean(axis=0)
    array_img = nb.Nifti1Image(avg_isc,dim_sub.affine,dim_sub.header)
    nb.save(array_img,isc_outs+task+"/"+task+"_average_isc.nii.gz")